In [ ]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [ ]:
#| export
from decimal import Decimal, getcontext
from typing import Union
import gc

import pandas as pd
import pyodbc
from rich.console import Console
from pyarrow import ArrowInvalid, ArrowTypeError
from fastcore.xtras import Path
from fastcore.test import test_eq
from fastcore.foundation import L
from tqdm.auto import tqdm
import pyodbc
from pymongo import MongoClient

from extracao.constants import *
from extracao.format import parse_bw, format_types, input_coordenates

getcontext().prec = 5

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [ ]:
#| export
def connect_db(server: str = 'ANATELBDRO05', # Servidor do Banco de Dados
               database: str = 'SITARWEB', # Nome do Banco de Dados
               trusted_conn: str = 'yes', # Conexão Segura: yes | no
               mult_results: bool = True, # Múltiplos Resultados
              )->pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )

In [ ]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (SQL_RADCOM,SQL_STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [ ]:
#| export
def clean_mosaico(df: pd.DataFrame, # DataFrame com os dados de Estações e Plano_Básico mesclados 
                pasta: Union[str, Path], # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame: # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    # df = input_coordenates(df, pasta) # TODO: Implementar função de verificação de coordenadas diretamente no arquivo final base e eliminar essa chamada
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.loc["Frequência"] = df.Frequência.str.replace(",", ".")    
    df = df[df.Frequência.notna()].reset_index(drop=True)
    df.loc["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Num_Serviço == "205", "Frequência"] = df.loc[
        df.Num_Serviço == "205", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df

## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [ ]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = format_types(df, stem)
    df = df.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
    df = df.drop_duplicates(keep='first').reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError):
        file.unlink(missing_ok=True)
        try:
            file = Path(f"{folder}/{stem}.fth")
            df.to_feather(file)
        except (ArrowInvalid, ArrowTypeError):
            file.unlink(missing_ok=True)
            try:
                file = Path(f"{folder}/{stem}.xlsx")
                with pd.ExcelWriter(file) as wb:
                    df.to_excel(
                        wb, sheet_name="DataBase", engine="openpyxl", index=False
                    )
            except Exception as e:
                raise ValueError(f"Could not save {stem} to {file}") from e
    return df



In [ ]:
# | export
def update_radcom(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder: Union[str, Path] # Pasta onde salvar os arquivos
        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:            
            df = pd.read_sql_query(SQL_RADCOM, conn)
            return _save_df(df, folder, "radcom")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [ ]:
#| eval: false
import warnings
warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')

In [ ]:
#| eval: false
folder = Path.cwd().parent / 'dados'
conn = connect_db()
radcom = update_radcom(conn, folder)
radcom

c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support 
SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects 
are not tested, please consider using SQLAlchemy
  warnings.warn(

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,1008210959,Turilândia,2112456,MA,-2.228611111111,-45.306666666666665,P
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,699491851,Alvorada de Minas,3102407,MG,-18.734166666666667,-43.36472222222217,3
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,699359830,Congonhas do Norte,3118106,MG,-18.812777777777665,-43.673611111111,3
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,690859562,Dom Joaquim,3122603,MG,-18.95,-43.266666666666666,3
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,1008401606,Castanheira,5102850,MT,-11.137222222222167,-58.61333333333334,P-A
...,...,...,...,...,...,...,...,...,...,...
4925,107.9,ASSOCIAÇÃO CULT COMUNIT MORUMBI,50407431578,697707695,São José dos Campos,3549904,SP,-23.267777777777667,-45.8975,3-B
4926,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50413097013,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3
4927,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3
4928,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,693049723,São José dos Campos,3549904,SP,-23.191944444444335,-45.87527777777767,3


In [ ]:
#|export
def update_stel(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder:Union[str, Path] # Pasta onde salvar os arquivos        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="bouncingBall",
    ) as status:
        try:            
            df = pd.read_sql_query(SQL_STEL, conn)
            return _save_df(df, folder, "stel")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [ ]:
#| eval: false
stel = update_stel(conn, folder)
stel

c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support 
SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects 
are not tested, please consider using SQLAlchemy
  warnings.warn(

Output()

,Frequência,Classe_Emissão,Largura_Emissão,Classe,Num_Serviço,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Validade_RF
0,10512,F8W,18M0,RP,256,GLOBO COMUNICAÇÃO E PARTICIPAÇÕES S/A,01032381230,3979016,Rio de Janeiro,3304557,RJ,-22.919722222222166,-43.214722222222164,1997-03-05
1,10656,F8W,18M0,TX,256,GLOBO COMUNICAÇÃO E PARTICIPAÇÕES S/A,01032381230,3979008,Rio de Janeiro,3304557,RJ,-22.951388888888832,-43.237222222222165,1997-03-05
2,10835,D7D,40M0,FX,046,UNIVERSAL TELECOM S.A.,50014055481,688664008,Ferraz de Vasconcelos,3515707,SP,-23.536669444444332,-46.38000277777767,2029-02-17
3,10915,D7D,40M0,FX,046,UNIVERSAL TELECOM S.A.,50014055481,692056769,São Paulo,3550308,SP,-23.609975,-46.611322222222164,2029-02-17
4,10915,D7W,40M0,FX,046,Cataguases Net Ltda,50408658053,1008135523,Cataguases,3115300,MG,-24.412325,-42.685938888888835,2026-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68474,25.375,F9W,25K0,TX,060,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,50417771525,1008302110,Porto Alegre,4314902,RS,-30.010194444444334,-51.159388888888834,2039-04-08
68475,25.375,F9W,25K0,TX,060,REAL E BENEMERITA ASSOCIACAO PORTUGUESA DE BEN...,50404519970,1008765764,São Paulo,3550308,SP,-23.567138888888834,-46.64113888888883,2037-05-08
68476,25.375,F9W,25K0,TX,060,SOCIEDADE BENEFICENTE SAO CAMILO,50406930287,692391789,São Paulo,3550308,SP,-23.487777777777666,-46.627444444444336,2040-05-05
68477,25.375,F9W,25K0,TX,060,UNIMED PAULISTANA SOC COOPERATIVA DE TRABALHO ...,50407957189,696380056,São Paulo,3550308,SP,-23.562277777777666,-46.63858333333334,2026-09-01


In [ ]:
#|export
def update_mosaico(        
        mongo_client: MongoClient, # Objeto de conexão com o MongoDB
        folder: Union[str, Path] # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Efetua a query na tabela de Radiodifusão no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="clock"
    ) as status:  
        
        database = mongo_client["sms"]
        collection = database["srd"]
        query = {}
        list_data = list(collection.find(query, projection = MONGO_SRD))
        mosaico_df = pd.json_normalize(list_data)
        mosaico_df = mosaico_df.drop(columns=['estacao'])
        columns = list(COLS_SRD.keys())
        mosaico_df = mosaico_df[columns]
        mosaico_df.rename(COLS_SRD, axis=1, inplace=True)
        df = clean_mosaico(mosaico_df, folder)    
    return _save_df(mosaico_df, folder, "mosaico")

In [ ]:
#|eval: false
uri = input()
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [ ]:
#|eval: false
mosaico = update_mosaico(mongo_client, folder)
mosaico

Output()

c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\pandas\core\indexing.py:2115: FutureWarning: In a future
version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series 
behavior)
  new_ix = Index(new_ix)

In [ ]:
#| export
def update_telecom(mongo_client: MongoClient, # Objeto de conexão com o MongoDB
                   folder: Union[str, Path] # Pasta onde salvar os arquivos
)-> pd.DataFrame: # DataFrame com os dados atualizados
    """Efetua a query na tabela `licenciamento` no banco mongoDB `mongo_client` e atualiza o arquivo local"""


    console = Console()
    with console.status(
        "Consolidando os dados do Licenciamento...", spinner="clock"
    ) as status:

        database = mongo_client["sms"]
        collection = database["licenciamento"]    
        c = collection.find(MONGO_TELECOM, projection={k:1.0 for k in COLS_TELECOM.keys()})
        result = L()
        for doc in tqdm(c):
            result.append(doc)
        df = pd.json_normalize(result)
        df.drop('_id', axis=1, inplace=True)
        df.rename(COLS_TELECOM, axis=1, inplace=True)
        df['Designacao_Emissão'] = df.Designacao_Emissão.str.replace(',', ' ')
        df['Designacao_Emissão'] = df.Designacao_Emissão.str.strip().str.lstrip().str.rstrip().str.upper()
        df['Designacao_Emissão'] = df.Designacao_Emissão.str.split(' ')
        df = df.explode('Designacao_Emissão')
        df.loc[df.Designacao_Emissão == '/', 'Designacao_Emissão'] = ''
        df.loc[:, ['BW(kHz)', 'Classe_Emissão']]  = df.Designacao_Emissão.apply(parse_bw).tolist()
        df.drop('Designacao_Emissão', axis=1, inplace=True)
        subset = ['Entidade', 'Longitude', 'Latitude', 'Classe', 'Frequência', 'Num_Serviço', 'BW(kHz)', 'Classe_Emissão']
        df_sub = df[~df.duplicated(subset=subset, keep='first')].reset_index(drop=True).copy()
        df_sub = df_sub.set_index(subset).sort_index()
        df_sub['Count'] = (df.groupby(subset).count()['Número_Estação']).tolist()
        df_sub['Count'] = df_sub['Count'].astype('string')
        df_sub.loc[df_sub.Count != '1', 'Número_Estação'] = df_sub.loc[df_sub.Count != '1', 'Número_Estação'] + '+' + df_sub.loc[df_sub.Count != '1', 'Count']
        df_sub.drop('Count', axis=1, inplace=True)
        del df ; gc.collect()
        df_sub = df_sub.reset_index()
    return _save_df(df_sub, folder, 'telecom')

In [ ]:
#| eval: false
# telecom = update_telecom(mongo_client, folder)

In [ ]:
# | export
def valida_coord(
    conn: pyodbc.Connection, # Objeto de conexão de banco    
    row
) -> pd.DataFrame: # DataFrame com dados do município

    # sql_params = (row['cod_municipio'], row['latitude'], row['longitude'])        
    # df = pd.read_sql(SQL_VALIDA_COORD, conn, params = sql_params)
    if row['Código_Município'].strip():
        
        row['Longitude'] = row['Longitude'] if row['Longitude'] else '0'
        row['Latitude'] = row['Latitude'] if row['Latitude'] else '0'

        sql = SQL_VALIDA_COORD.format(row['Longitude'], row['Latitude'], row['Código_Município'])
        # print(sql)
        crsr = conn.cursor()
        crsr.execute(sql)
        result = crsr.fetchone()
        # print(result)
        if result == None:
            return (row['Município'], row['Longitude'], row['Latitude'], 9)
        elif result.COORD_VALIDA == 1:
            return result
        else:
            return (result.NO_MUNICIPIO, result.NU_LONGITUDE, result.NU_LATITUDE, result.COORD_VALIDA)
    else:
        return (row['Município'], row['Longitude'], row['Latitude'], 9)

In [ ]:
# | export
def update_base(
    conn: pyodbc.Connection, # Objeto de conexão de banco
    clientMongoDB: MongoClient, # Ojeto de conexão com o MongoDB
    folder: Union[str, Path] # Pasta onde salvar os arquivos    
) -> pd.DataFrame: # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder, ).loc[:, TELECOM]
    radcom = update_radcom(conn, folder).loc[:, SRD]
    mosaico = update_mosaico(clientMongoDB, folder).loc[:, RADIODIFUSAO]    
    telecom = update_telecom(clientMongoDB, folder)

    # Filtrando RADCOM
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = pd.NA
    radcom["BW(kHz)"] = '256'
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Validade_RF"] = pd.NA
    radcom["Fonte"] = "SRD"

    # Filtrando STEL
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    stel.loc[:, ['BW(kHz)', 'Classe_Emissão']] = stel.Largura_Emissão.fillna('').apply(parse_bw).tolist()
    stel.loc[stel.Classe_Emissão == '', 'Classe_Emissão'] = pd.NA
    stel.drop('Largura_Emissão', axis=1, inplace=True)

    # Filtrando MOSAICO
    mosaico["Fonte"] = "MOS"
    mosaico.loc[:, ['BW(kHz)', 'Classe_Emissão']] = mosaico.Num_Serviço.map(BW_MAP).apply(parse_bw).tolist()
    mosaico.loc[mosaico.Classe_Emissão == '', 'Classe_Emissão'] = pd.NA

    # Filtrando LICENCIAMENTO
    telecom["Fonte"] = 'LIC'

    rd = (
        # pd.concat([mosaico, radcom, stel])
        pd.concat([mosaico, radcom, stel, telecom])        
            .sort_values(["Frequência", "Latitude", "Longitude"])
            .reset_index(drop=True)
    )
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)


    # Verificando conteúdo do CSV Final
    # filepath = Path(f"\\\\servrepds\\dw$\\Input\\sentinela\\update_base.csv") 
    # rd.to_csv(filepath)

    # Validando Coordenadas 
    rd['coord_valida'] = None
    rd[['Município', 'Longitude', 'Latitude', 'coord_valida']] = rd.apply(lambda row: pd.Series(list(valida_coord(conn, row))), axis=1)
    rd = rd.drop(rd[rd.coord_valida == 9].index)

    return _save_df(rd, folder, "base")

[autoreload of extracao.reading failed: Traceback (most recent call last):
  File "c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "c:\Users\rsilva\Miniconda3\envs\anateldb\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "g:\OneDrive - ANATEL\anateldb\extracao\reading.py", line 16, in <module>
    from extracao.updates import (
ImportError: cannot import name 'update_licenciamento' from 'extracao.updates' (g:\OneDrive - ANATEL\anateldb\extracao\updates.py)
]


In [ ]:
#| eval: false
base = update_base(conn, mongo_client, folder)
base

c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support 
SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects 
are not tested, please consider using SQLAlchemy
  warnings.warn(

Output()

c:\Users\rsilva\Miniconda3\envs\anateldb\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support 
SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects 
are not tested, please consider using SQLAlchemy
  warnings.warn(

Output()

Output()

0it [00:00, ?it/s]

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Validade_RF,Status,Fonte,BW(kHz),Classe_Emissão,Data_de_Validade,Count
0,0.028,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,,3303500,RJ,-22.662778,-43.476389,OP,019,<NA>,LIC-LIC-01,LIC,8.0,J9E,2033-08-17,1
1,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,,3103751,MG,-18.41,-49.1,OP,019,<NA>,LIC-LIC-01,LIC,1.0,J3E,2033-08-17,1
2,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,,3509502,SP,-22.774167,-47.004444,OP,019,<NA>,LIC-LIC-01,LIC,1.0,J3E,2033-08-17,1
3,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,,3304557,RJ,-22.926667,-43.265,OP,019,<NA>,LIC-LIC-01,LIC,0.5,J3E,2033-08-17,1
4,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,,3550308,SP,-23.441667,-46.590833,OP,019,<NA>,LIC-LIC-01,LIC,1.0,J3E,2033-08-17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787550,990.0,RADIO CULTURA DE DOIS CORREGOS LTDA,02008013766,7804318,Dois Córregos,3514106,SP,-22.3666666666666666,-48.3666666666666666,B,205,2024-05-01,AM-C3,MOS,10.0,<NA>,<NA>,<NA>
787551,990.0,RADIO CONTEMPORANEA LTDA,01008011029,5534682,Rio de Janeiro,3304557,RJ,-22.7625000000000000,-43.0083333333333333,A,205,2028-10-04,AM-C3,MOS,10.0,<NA>,<NA>,<NA>
787552,990.0,RADIO CLUBE PEDRO OSORIO LTDA,03022119852,9623450,Pedro Osório,4314209,RS,-31.8527777777776666,-52.8000000000000000,C,205,2025-10-24,AM-C3,MOS,10.0,<NA>,<NA>,<NA>
787553,990.0,FUNDACAO SANTA LUZIA DE MOSSORO,07008009569,322688582,Mossoró,2408003,RN,-5.2400000000000000,-37.3138888888888333,B,205,2023-11-01,AM-C4,MOS,10.0,<NA>,<NA>,<NA>


In [ ]:
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# req = Request(ESTACAO)
# try:
#     response = urlopen(req)
# except URLError as e:
#     if hasattr(e, 'reason'):
#         print('We failed to reach a server.')
#         print('Reason: ', e.reason)
#     elif hasattr(e, 'code'):
#         print('The server couldn\'t fulfill the request.')
#         print('Error code: ', e.code)
# else:
#     Path.cwd().joinpath('estações.zip').write_bytes(response.read())